## 2021 Day 16

https://adventofcode.com/2021/day/16

In [1]:
from functools import reduce

In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
inp = open('input-16.txt').read()

In [4]:
test_lit = 'D2FE28'

test_op_lt0 = '38006F45291200'
test_op_lt1 = 'EE00D40C823060'

test_inp1 = '8A004A801A8002F478'
test_inp2 = '620080001611562C8802118E34'
test_inp3 = 'C0015000016115A2E0802F182340'
test_inp4 = 'A0016C880162017C3686B18A3D4780'

In [5]:
def hex2bin(inp):
    out = ''.join([f'{int(c, base=16):04b}' for c in list(inp)])
    return out

In [6]:
class Packet:
    def __len__(self):
        return len(self.code)

def parse_packet(code, depth=0, verbose=0):
    
    def log(l, *a, **kw):
        prefix = '  ' * depth
        if l <= verbose:
            print(prefix, end='')
            print(*a, **kw)
            
    if any([c not in '01' for c in code]):
        code = hex2bin(code)
    orig_code = code
            
    version, code = int(code[:3], base=2), code[3:]
    typ, code = int(code[:3], base=2), code[3:]
    log(1, f'depth = {depth}, version = {version}, type = {typ}')
    log(1, f'code = {orig_code}')
    if typ == 4:
        full_content = ''
        while True:
            prefix, content, code = code[0], code[1:5], code[5:]
            full_content += content
            log(2, f'full_content =', full_content)
            log(2, f'->len(orig_code), len(code) = {len(orig_code)}, {len(code)})')
            if prefix == '0':
                break
        value = int(full_content, base=2)
        used_length = len(orig_code) - len(code)
        log(2, f'->used_length = {used_length}')
        return Literal(version, typ, value, code=orig_code[:used_length]), code
    
    length_type, code = int(code[:1]), code[1:]
    subpackets = []
    if length_type == 0:
        subpackets_bits, code = int(code[:15], base=2), code[15:]
        length_info = subpackets_bits
        log(2, f'length_type = {length_type}, subpackets_bits = {subpackets_bits}')
        while True:
            subpacket, code = parse_packet(code, depth=depth+1, verbose=verbose)
            log(2, f'->subpacket = {subpacket}')
            log(2, f'->remaining_code = {code}')
            subpackets.append(subpacket)
            log(2, f'->len(orig_code), len(code) = {len(orig_code)}, {len(code)})')
            log(2, len(orig_code) - len(code) - 7, subpackets_bits)
            if len(orig_code) - len(code) - 7-15 >= subpackets_bits:
                break
                
    elif length_type == 1:
        subpackets_count, code = int(code[:11], base=2), code[11:]
        length_info = subpackets_count
        log(2, f'length_type = {length_type}, subpackets_count = {subpackets_count}')
        for i in range(subpackets_count):
            subpacket, code = parse_packet(code, depth=depth+1, verbose=verbose)
            log(2, f'->subpacket = {subpacket}')
            log(2, f'->remaining_code = {code}')
            log(2, f'->len(orig_code), len(code) = {len(orig_code)}, {len(code)})')
            subpackets.append(subpacket)
    else:
        raise RuntimeError(f'illegal length_type = {length_type}')
            
    used_length = len(orig_code) - len(code)
    out = Operator(version, typ, length_type, length_info, subpackets, code=orig_code[:used_length])
    if depth:
        return out, code
    else:
        return out

class Literal(Packet):
    def __init__(self, version, typ, value, code):
        self.version = version
        self.typ = typ
        self.value = value
        self.code = code
        self.subpackets = []
        
    def __repr__(self):
        return f'Literal(v{self.version},l{len(self)};{self.value})'
    
    
class Operator(Packet):
    
    operations = {
        0: sum,
        1: lambda x: reduce((lambda a,b: a*b), x),
        2: min,
        3: max,
        5: lambda x: 1 if x[0] > x[1] else 0,
        6: lambda x: 1 if x[0] < x[1] else 0,
        7: lambda x: 1 if x[0] == x[1] else 0,
    }
    
    def __init__(self, version, typ, length_type, length_info, subpackets, code):
        self.version = version
        self.typ = typ
        self.length_type = length_type
        self.length_info = length_info
        self.subpackets = subpackets
        self.code = code
        
    def __repr__(self):
        subpackets = ','.join([str(p) for p in self.subpackets])
        return f'Operator(v{self.version},t{self.typ},lt{self.length_type},li{self.length_info};{subpackets})'
    
    @property
    def value(self):
        return self.operations[self.typ]([subpacket.value for subpacket in self.subpackets])

In [7]:
x = parse_packet(test_inp2, verbose=0)
print(x)

Operator(v3,t0,lt1,li2;Operator(v0,t0,lt0,li22;Literal(v0,l11;10),Literal(v5,l11;11)),Operator(v1,t0,lt1,li2;Literal(v0,l11;12),Literal(v3,l11;13)))


## Part 1

In [8]:
def version_sum(packet):
    return packet.version + sum(version_sum(p) for p in packet.subpackets)

In [9]:
for t in test_inp1, test_inp2, test_inp3, test_inp4:
    print(version_sum(parse_packet(t)))

16
12
23
31


In [10]:
version_sum(parse_packet(inp))

999

## Part 2

In [ ]:
parse_packet(inp).value